## Hybrid Search Langchain

In [11]:
!pip install --upgrade --quite pinecone-client pinecone-text pinecone-notebooks


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --quite


In [12]:
import sys
print(sys.executable)


c:\Users\Divya\conda_envs\venv\python.exe


In [13]:
!"{sys.executable}" -m pip install pinecone



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from pinecone import Pinecone, ServerlessSpec


In [16]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [17]:
import os 
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone" 
##initialize pinecone client
pc=Pinecone(api_key=api_key)


##create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  ##dimension of dense vector 
        metric= "dotproduct", ##Sparse values supported only for dotproduct metric
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

In [18]:
index=pc.Index(index_name)
index

In [19]:
##Vector embedding and sparse matrix 

import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [3]:
import sys
!{sys.executable} -m pip install pinecone-text



  Attempting uninstall: mmh3

    Found existing installation: mmh3 5.1.0

    Uninstalling mmh3-5.1.0:

   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   ---------------------------------------- 0/3 [mmh3]
   -----------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Divya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [20]:
sentences=[
    "IN 2023, I Visited Paris",
    "In 2022,I visited New York",
    "In 2021 , I visted New Australia",
]

##tfidf values on these sentences
bm25_encoder.fit(sentences)


##store values to json file 
bm25_encoder.dump("bm25_encoder.json")

#load to your BM25encoder object
bm25_encoder=BM25Encoder().load("bm25_encoder.json")

100%|██████████| 3/3 [00:00<00:00, 2966.97it/s]


In [21]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [22]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000204251ECE60>, index=<pinecone.db_data.index.Index object at 0x00000204165E8230>)

In [23]:
retriever.add_texts(
[
    "IN 2023, I Visited Paris",
    "In 2022,I visited New York",
    "In 2021 , I visted New Australia",
]
)

100%|██████████| 1/1 [00:02<00:00,  2.24s/it]


In [24]:
retriever.invoke("What city did i visit last")

[Document(metadata={'score': 0.284365773}, page_content='In 2022,I visited New York'),
 Document(metadata={'score': 0.267601669}, page_content='IN 2023, I Visited Paris'),
 Document(metadata={'score': 0.141474247}, page_content='In 2021 , I visted New Australia')]